In [2]:
import os
import time
import csv
import re
import codecs
import contextlib
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException

### setup area

In [3]:
base = 1
nomor = 1
review_count = 0

In [4]:
jumlah_review = 25 

In [5]:
read_filename = "names.csv"
write_filename = "review1.csv"

read_path =os.path.join("response", read_filename)
write_path = os.path.join("review",write_filename)

## sub function

In [6]:
def changerange(basic):
    global nomor
    nomor = basic + 3

In [7]:
def clean_filename(filename):
    global cleaned_filename
    # Remove special characters using regular expressions
    cleaned_filename = re.sub(r'[^\w\s.-]', '', filename)

In [8]:
def separate_string(string):
    global experience
    global num_of_review
    pattern = r'^(.*?)\s·\s(.*)$'
    match = re.match(pattern, string)
    if match:
        experience = match.group(1).strip()
        num_of_review = match.group(2).strip()
    else:
        experience = '-'
        num_of_review = string.strip()

## Google maps function

In [9]:
def search(name):
    global driver , actions
    global link
    
    link = "https://www.google.com/maps/search/{} jogja".format(name)
    
    #open up chrome and stuff
    driver = webdriver.Chrome()
    driver.get(link)
    actions = ActionChains(driver)
    
    #kalau gak langsung ketemu
    
    try:
        restaurant_link = driver.find_elements(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a')
        
        for links in restaurant_link:
            link = links.get_attribute("href")
            driver.get(link)
    except:
        pass
        
    


In [10]:
def review_button():
    # Ulasan button
    review_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]')))
    actions.move_to_element(review_button)
    actions.click().perform()

    # Click on the target element
    target_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[7]/div[1]/div[1]/div[1]')))
    target_element.click()

    # Press the down arrow key multiple times
    for _ in range(1500):  # Adjust the number of times to press the down arrow key as needed
        actions.send_keys(Keys.ARROW_DOWN).perform()
        
        
    time.sleep(10)

## scrapping function

In [11]:
def count_stars(stars_xpath):
    global empty_star_count
    global filled_star_count
    
    empty_star_count = 0
    filled_star_count = 0
    base_star = 0
    empty_star_class = "https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_empty_14.png"
    filled_star_class = "https://maps.gstatic.com/consumer/images/icons/2x/ic_star_rate_14.png"
    
    for i in range(5):
        base_star += 1
        final_xpath = '{}[{}]'.format(stars_xpath, base_star)
        
        try:
            stars = driver.find_elements(By.XPATH, final_xpath)

            for star in stars:
                class_name = star.get_attribute("src")

                if class_name == empty_star_class:
                    empty_star_count += 1
                elif class_name == filled_star_class:
                    filled_star_count += 1
        except Exception as e:
            print('An error occurred during star counting:', str(e))
    
    return empty_star_count, filled_star_count

In [12]:
delay = 1
def review_scrap():
    global name, time_of_review, num_of_review, experience, review, filled_star_count, review_count, xpath_name

    xpath_review = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[{}]/div/div/div[4]/div[2]/div/span[1]'.format(
        str(nomor))
    xpath_name = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[{}]/div/div/div[2]/div[2]/div[1]/button/div[1]'.format(
        str(nomor))
    xpath_button = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[{}]/div/div/div[4]/div[2]/div/span[2]/button'.format(
        str(nomor))
    xpath_star = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[{}]/div/div/div[4]/div[1]/span[1]/img'.format(
        str(nomor))
    xpath_exp = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[{}]/div/div/div[2]/div[2]/div[1]/button/div[2]'.format(
        str(nomor))
    xpath_time = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[9]/div[{}]/div/div/div[4]/div[1]/span[2]'.format(
        str(nomor))

    # show more review
    try:
        maximize_button = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, xpath_button)))
        actions.move_to_element(maximize_button)
        actions.click().perform()
    except:
        try:
            xpath_button = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[{}]/div/div/div[4]/div[2]/div/span[2]/button'.format(
                str(nomor))
            maximize_button = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, xpath_button)))
            actions.move_to_element(maximize_button)
            actions.click().perform()
        except:
            pass

    # just believe me when I type this, somehow they change sometimes AAAAAAAAAAAAAAAAAAAAA
    try:
        raw_name = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, xpath_name)))
        raw_time = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, xpath_time)))
    except:
        xpath_name = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[{}]/div/div/div[2]/div[2]/div[1]/button/div[1]'.format(
            str(nomor))
        
        xpath_star = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[{}]/div/div/div[4]/div[1]/span[1]/img'.format(
            str(nomor))
        
        raw_name = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, xpath_name)))

        xpath_time = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[{}]/div/div/div[4]/div[1]/span[2]'.format(
            str(nomor))
        
        raw_time = WebDriverWait(driver,  delay).until(EC.presence_of_element_located((By.XPATH, xpath_time)))

    name = raw_name.text
    time_of_review = raw_time.text

    try:
        global experience, num_of_review

        try:
            raw_exp = WebDriverWait(driver,  delay).until(EC.presence_of_element_located((By.XPATH, xpath_exp)))
            raw_review = WebDriverWait(driver,  delay).until(EC.presence_of_element_located((By.XPATH, xpath_review)))
        except:
            xpath_exp = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[{}]/div/div/div[2]/div[2]/div[1]/button/div[2]'.format(
                str(nomor))
            raw_exp = WebDriverWait(driver,  delay).until(EC.presence_of_element_located((By.XPATH, xpath_exp)))

            xpath_review = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/div[{}]/div/div/div[4]/div[2]/div/span[1]'.format(
                str(nomor))
            raw_review = WebDriverWait(driver,  delay).until(EC.presence_of_element_located((By.XPATH, xpath_review)))

        exp = raw_exp.text

        separate_string(exp)
        review = raw_review.text
    except:
        review = ''
        experience = ''
        num_of_review = ''

    count_stars(xpath_star)

    review_count += 1

    # Uncomment the print statements below for debugging purposes
#     print("====================")
#     print(f"Reviewer: {name}")
#     print(f"time of review : {time_of_review} ")
#     print(f"exp: {experience}")
#     print(f"num: {num_of_review}")
#     print(f"Review: {review}")
#     print(f"Star:{filled_star_count}")
#     print(f"empty Star:{empty_star_count}")

### csv file setup

In [13]:
# Define the fieldnames for the CSV file
fieldnames = ['PLACE NAME','LINK','TIME', 'NAME', 'EXPERIENCE', 'NUMBER OF REVIEW', 'REVIEW', 'STAR']

# RUN THIS

In [16]:
driver = webdriver.Chrome()
# Create the 'review' directory if it doesn't exist
os.makedirs('review', exist_ok=True)

with open(read_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)

    next(reader)
    for row in reader:
        clean_filename(row[0])
        NAMA = cleaned_filename
        nomor = 1
        review_count = 0

        # Generate the write file path based on the restaurant name
        write_filename = f"{NAMA}.csv"
        write_path = os.path.join("review", write_filename)

        # Check if the file already exists and have more than 20 lines
        if os.path.exists(write_path):
            with open(write_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                non_empty_lines = [line for line in lines if line.strip()]  # Filter non-empty lines
                if len(non_empty_lines) > 20:
                    print(f"{NAMA} is already scrapped. Check or delete the file if you want to rescrape it.")
                    driver.quit()  # Close the webdriver
                    continue
        try:
            search(NAMA)
        except:
            continue
            
        try:
            review_button()
        except:
            continue
        
        # Open the CSV file for writing with UTF-8 encoding
        with codecs.open(write_path, mode='w', encoding='utf-8', errors='ignore') as details_file:
            writer = csv.DictWriter(details_file, fieldnames=fieldnames)
            writer.writeheader()

            # Write the name and link as separate rows
            writer.writerow({'PLACE NAME': NAMA})
            writer.writerow({'LINK': link})
            writer.writerow({})  # Empty row for separation

            # Write the column headers
            writer.writerow({
                'TIME': 'TIME',
                'NAME': 'NAME',
                'EXPERIENCE': 'EXPERIENCE',
                'NUMBER OF REVIEW': 'NUMBER OF REVIEW',
                'REVIEW': 'REVIEW',
                'STAR': 'STAR'
            })

            # Use tqdm to create a progress bar
            progress_bar = tqdm(range(jumlah_review), desc=f"Scraping reviews for {NAMA}")

            for x in progress_bar:
                try:
                    review_scrap()
                    changerange(nomor)
                except:
                    pass

                details_dict = {
                    'TIME': time_of_review,
                    'NAME': name,
                    'EXPERIENCE': experience,
                    'NUMBER OF REVIEW': num_of_review,
                    'REVIEW': review,
                    'STAR': filled_star_count
                }

                writer.writerow(details_dict)

                # Update the progress bar
                progress_bar.set_postfix({'Review': x + 1})

            # Close the progress bar
            progress_bar.close()

LOKO CAFE STASIUN TUGU is already scrapped. Check or delete the file if you want to rescrape it.
starting scraping for:TUGU LOR


Scraping reviews for TUGU LOR: 100%|████████████████████████████████████████| 25/25 [00:16<00:00,  1.55it/s, Review=25]


Review scraping finished for TUGU LOR
Ramu Nostalgia - Tugu Yogyakarta is already scrapped. Check or delete the file if you want to rescrape it.
starting scraping for:Kedai Kolega Tugu


Scraping reviews for Kedai Kolega Tugu: 100%|███████████████████████████████| 25/25 [00:36<00:00,  1.47s/it, Review=25]


Review scraping finished for Kedai Kolega Tugu
starting scraping for:Wedangan


...

Review scraping finished for Pos ronda rt 06 kajar 2
starting scraping for:Shafea home
starting scraping for:KEDAI KAPTEN MUDA


Scraping reviews for KEDAI KAPTEN MUDA: 100%|███████████████████████████████| 25/25 [01:41<00:00,  4.07s/it, Review=25]


Review scraping finished for KEDAI KAPTEN MUDA
starting scraping for:KOPIYUK
starting scraping for:Karaoke Edy
starting scraping for:Ralls  Dwy


Scraping reviews for Ralls  Dwy: 100%|██████████████████████████████████████| 25/25 [01:42<00:00,  4.10s/it, Review=25]

Review scraping finished for Ralls  Dwy


In [ ]:
driver = webdriver.Chrome()
# Create the 'review' directory if it doesn't exist
os.makedirs('review', exist_ok=True)

# Open the input file and loop through each vessel name
with open(read_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row

    rows = list(reader)  # Convert the remaining rows to a list
    total_rows = len(rows)  # Get the total number of rows

    for row in tqdm(rows, total=total_rows, desc="Processing rows"):
        clean_filename(row[0])
        NAMA = cleaned_filename
        nomor = 1
        review_count = 0

        # Generate the write file path based on the restaurant name
        write_filename = f"{NAMA}.csv"
        write_path = os.path.join("review", write_filename)

        # Check if the file already exists and has more than 20 lines
        if os.path.exists(write_path):
            with open(write_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                non_empty_lines = [line for line in lines if line.strip()]  # Filter non-empty lines
                if len(non_empty_lines) > 20:
                    print(f"{NAMA} is already scrapped. Check or delete the file if you want to rescrape it.")
                    driver.quit()  # Close the webdriver
                    continue
        try:
            search(NAMA)
        except:
            continue

        try:
            review_button()
        except:
            continue

        # Open the CSV file for writing with UTF-8 encoding
        with codecs.open(write_path, mode='w', encoding='utf-8', errors='ignore') as details_file:
            writer = csv.DictWriter(details_file, fieldnames=fieldnames)
            writer.writeheader()

            # Write the name and link as separate rows
            writer.writerow({'PLACE NAME': NAMA})
            writer.writerow({'LINK': link})
            writer.writerow({})  # Empty row for separation

            # Write the column headers
            writer.writerow({
                'TIME': 'TIME',
                'NAME': 'NAME',
                'EXPERIENCE': 'EXPERIENCE',
                'NUMBER OF REVIEW': 'NUMBER OF REVIEW',
                'REVIEW': 'REVIEW',
                'STAR': 'STAR'
            })

            # Use tqdm to create a progress bar
            progress_bar = tqdm(range(jumlah_review), desc=f"Scraping reviews for {NAMA}")

            for x in progress_bar:
                try:
                    review_scrap()
                    changerange(nomor)
                except:
                    pass

                details_dict = {
                    'TIME': time_of_review,
                    'NAME': name,
                    'EXPERIENCE': experience,
                    'NUMBER OF REVIEW': num_of_review,
                    'REVIEW': review,
                    'STAR': filled_star_count
                }

                writer.writerow(details_dict)

                # Update the progress bar
                progress_bar.set_postfix({'Review': x + 1})

            # Close the progress bar
            progress_bar.close()
        progress_bar.close()

            print(f"Review scraping finished for {NAMA}")
